In [ ]:
pip install -U dashscope

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

abc_txt = os.environ.get('SERVER_LESS_KEY')
print(abc_txt)

In [ ]:
## 聊天
import os
import dashscope

message = [
    {"role":"system", "content": "You are a helpful assistant."},
    {"role":"user", "content": "介绍一下你自己"}
]
response = dashscope.Generation.call(
    api_key = abc_txt,
    model="qwen-plus",
    messages=message,
    result_format="message",
    stream=True
)

# print(list(response))

for chunk in response:
    print(chunk)
    if chunk.get("status_code") == 200:
        # content = chunk.get("output").get("choices")[0].get("message")
        # content = chunk.output.choices[0].message
        content = chunk['output']['choices'][0]['message']
        print(content, end="", flush=True)
    else:
    # 如果出现错误，打印错误信息
        print(f"错误码：{chunk.code}")
        print(f"错误信息：{chunk.message}")
        break

In [ ]:
## 多轮对话

import os
import dashscope
from dashscope import Generation

api_key = os.getenv("DASHSCOPE_API_KEY")

def get_response(input):
    response = Generation.call(
        api_key = api_key,
        model = "qwen-plus",
        messages = input,
        result_format = "message"
    )
    return response

chat_messages = [
    {
        "role": "system",
        "content": """你是一名阿里云百炼手机商店的店员，你负责给用户推荐手机。手机有两个参数：屏幕尺寸（包括6.1英寸、6.5英寸、6.7英寸）、分辨率（包括2K、4K）。
        你一次只能向用户提问一个参数。如果用户提供的信息不全，你需要反问他，让他提供没有提供的参数。如果参数收集完成，你要说：我已了解您的购买意向，请稍等。"""
    }
]

assitant_message = "欢迎光临阿里云百炼手机商店，您需要购买什么尺寸的手机呢？"

print(f"模型输出：{assitant_message} \n")

while "我已了解您的购买意向" not in assitant_message:
    user_input = input("用户输入：")
    chat_messages.append({
        "role":"user",
        "content": user_input
    })
    response = get_response(chat_messages)
    assitant_message = response.output.choices[0].message.content
    chat_messages.append({
        "role":"assistant",
        "content": assitant_message
    })
    print(f"模型输出：{assitant_message}")
else:
    response = get_response(chat_messages)
    assitant_message = response.output.choices[0].message.content
    print("已获取到用户的购买意向",assitant_message)


In [ ]:
## 流式输出
import os
from dashscope import Generation
api_key = os.getenv('SERVER_LESS_KEY')



response = Generation.call(
    api_key=api_key,
    model='qwen-plus',
    messages = [
        {
            "role": "user",
            "content": "请自我介绍"
        }
    ],
    result_format = "message",
    stream = True,
    incremental_output=True,

)

full_result = ""
for res in response:
    result_str = res.output.choices[0].message.content
    print(result_str)
    full_result += result_str

print(full_result)

In [ ]:
## 思考模式
import os
from dashscope import Generation

api_key = os.getenv('SERVER_LESS_KEY')

def chat(prompt):
    response = Generation.call(
        api_key=api_key,
        model='qwen-plus',
        messages = prompt,
        result_format = 'message',
        stream = True,
        incremental_output = True,
        enable_thinking = True
    )
    return response 


reasoning_content = ''
answer_content = ''

messages = [
    {
        "role": "user",
        "content": "你是谁？"
    }
]

response = chat(messages)

print("\n"+"=" * 20 + "思考过程" + "=" * 20)

if_thinking = True

for res in response:
    if ((not res.output.choices[0].message.reasoning_content) and (not res.output.choices[0].message.content)):
        pass
    else: 
        if(res.output.choices[0].message.reasoning_content and (not res.output.choices[0].message.content)):
            print(res.output.choices[0].message.reasoning_content, end="",flush=True)
            reasoning_content += res.output.choices[0].message.reasoning_content

            if_thinking = False # 结束思考标记

        elif(res.output.choices[0].message.content):

            # 在非思考模式标记下，输出一次完整回复
            if not if_thinking:
                print("\n" + "=" * 20 + "完整回复" + "=" * 20)
                # 将思考模式标记切换成 True
                if_thinking = True
                # 输出回复内容
            print(res.output.choices[0].message.content, end="",flush=True)
            answer_content += res.output.choices[0].message.content
    

In [ ]:
## 开启搜索模式

import os
from dashscope import Generation

api_key = os.getenv('SERVER_LESS_KEY')

messages = [
    {
        "role": "user",
        "content": "2025年6月，在“午夜之锤”行动中美国出动了几架B2轰炸机？"
    }
]

def chat(prompt):
    response = Generation.call(
        api_key=api_key,
        model = 'qwen-plus',
        messages = prompt,
        result_format = 'message',
        stream = True,
        enable_thinking = True,
        incremental_output = True,
        enable_search = True, # 开启联网搜索
        search_options = {
            "forced_search": True, # 强制开启联网搜索
            "enable_source": True, # 使返回结果包含搜索来源的信息，OpenAI 兼容方式暂不支持返回
            "enable_citation": True, # 开启角标标注功能
            "citation_format": "[ref_<number>]", # 角标形式为[ref_i]
            "search_strategy": "pro" # 模型将搜索10条互联网信息
        }
    )
    return response


reasoning_content = ""
answer_content = ""

is_first_res = True

is_reasoning = True


print("\n" + "="*20 + "搜索信息" + "="*20)

response = chat(messages)

for res in response:
    # 判断是否为第一个chunk，便于打印搜索信息
    if is_first_res:
        search_content = res.output.search_info['search_results']
        is_first_res = False
        for info in search_content:
            print(f"{info['index']}. 来源：{info['site_name']}, {info['title']}, 链接：{info['url']}")
    
    # 输出思考过程，首先判断当前message.reasoning_content中是否有内容，且message.content无内容时为思考模式
    if ((res.output.choices[0].message.reasoning_content) and (not res.output.choices[0].message.content)):
        # 打印一次正在思考的提示，然后将is_reasoning标记置为False，避免下一次循环时重复打印正在思考的提示
        if is_reasoning:
            print("\n"+"=" * 20 + "思考过程" + "=" * 20)
            is_reasoning = False
        print(res.output.choices[0].message.reasoning_content,end="",flush=True)
        reasoning_content += res.output.choices[0].message.reasoning_content
    elif (res.output.choices[0].message.content):
        # 打印一次正在输出结果的提示，然后将is_reasoning标记置为True，避免下次循环时重复打印正在输出结果的提示
        if not is_reasoning:
            print("\n"+"=" * 20 + "输出结果" + "=" * 20)
            is_reasoning = True
        print(res.output.choices[0].message.content,end="",flush=True)
        answer_content += res.output.choices[0].message.content






In [ ]:
## 工具调用

import os
from dashscope import Generation
import random
from datetime import datetime

import json

api_key = os.getenv('SERVER_LESS_KEY')


# 自定义获取时间的工具
def get_current_time():
    time_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"当前的时间是{time_str}")
    return f"当前的时间是{time_str}"


# 自定义获取天气的工具
def get_local_weather(arguments):
    weather_list = ['晴天','多云','雨天']

    weather = random.choice(weather_list)

    location = arguments['location']

    print(f"{location}的天气是{weather}")
    return f"{location}的天气是{weather}"

# 测试工具函数
# get_local_weather({'location':'上海'})
# get_current_time()


#创建tools列表

tools = [
    {
        "type":"function",
        "function":{
            "name":"get_current_time",
            "description":"当需要查询时间的时候，请调用此函数",
            "parameters":{}
        }
    },
    {
        "type":"function",
        "function":{
            "name":"get_local_weather",
            "description":"当需要查询天气的时候，请调用此函数",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"城市或者区县的名称，比如北京市、兴山县等等"
                    }
                },
                "required": ["location"],
            }
        }
    }
]

messages = [
    {
        "role":"system",
        "content":"你是一个智能助手，专门负责调用各种工具来帮助用户解决问题。请结合工具返回的结果给用户提供相关建议。"
    },
    {
        "role":"user",
        "content":"上海的天气下雨吗"
    }
]

# 验证创建工具列表
# tool_arr = [tool['function']['name'] for tool in tools]
# print(tool_arr)

def chat(prompt):
    response = Generation.call(
        api_key = api_key,
        model="qwen-plus",
        messages=prompt,
        tools=tools,
        result_format="message",
        # stream=True,

        # ！！ 开启思考模式，会改变模型函数调用信息的输出结构，模型会先输出思考内容再输出函数调用信息
        # enable_thinking = True,

        # 并行工具调用，支持输入问题需要调用多次工具
        # parallel_tool_calls=True,
    )
    return response

function_mapper = {
    "get_current_time":get_current_time,
    "get_local_weather":get_local_weather
}


def call_with_messages(prompt):
    
    res = chat(prompt)
    msg = res.output.choices[0].message

    # 大模型根据输出的tool_calls信息进行总结，必须得将msg加入到messages中
    messages.append(msg)


    print(json.dumps(res,ensure_ascii=False, indent=4))

    answer_content = ''

    if 'tool_calls' not in msg:
        print(f"直接回答问题：   {msg.content}")

    elif 'tool_calls' in msg:
        fn_name = msg.tool_calls[0]["function"]["name"]
        arguments = json.loads(msg.tool_calls[0]["function"]["arguments"])


        fn = function_mapper[fn_name]
        if fn_name == "get_current_time":
            answer_content = fn()
        else:
            answer_content = fn(arguments)
        
        # 大模型根据输出的tool_calls信息进行总结，必须得将函数调用的结果加入到messages中
        messages.append({
            "role":"tool",
            "name":fn_name,
            "content":answer_content
        })

call_with_messages(messages)

second_res =  chat(messages)
second_msg = second_res.output.choices[0].message.content
print('\n'+  '='*20 + '第二轮回答内容' + '='*20)
# print(second_res)
print(second_msg)